In [1]:
using Plots

In [2]:
@everywhere using DataFrames

In [34]:
rmprocs(5)

Task (done) @0x00000001227ab850

In [13]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [38]:
@everywhere begin
# returns a LxL Matrix filled randomly with ±1
function randomConfiguration(L::Int)
    return rand(Int8[-1,1], L,L)
end

# returns a configuration where all bonds are frustrated
function frustratedConfiguration(L::Int)
    return Int8[(-1)^(i+j) for i=1:L,j=1:L]
end

## Energy of configuration
## Arguments:
##           state::Matrix{Int8}
##  coupling J::Float64 defaults to 1.0
##  field    h::Float64 defaults to 0.0
function H(state::Matrix{Int8},J=1.,h=0.)
    L = size(state,1)
    s = h*(sum(state))
    
    @inbounds for i in 1:L, j in 1:L
        s += -J*state[i,j]*(state[i%L+1,j]+state[i,j%L+1])
    end
    return s
end

## Energy difference between the current stateuration `state`
## and one spin flipped at position `pos::Int`
## Arguments:
##           state::Matrix{Int8}
##  `pos` is the linear index of `state` 1..L^2
##  coupling J::Float64 defaults to 1.0
##  field    h::Float64 defaults to 0.0
function dH(state::Matrix{Int8},pos,J=1.,h=0.)
    L = size(state,1)
    i,j = ind2sub(state,pos) #convert the linear index to index pair (i,j)
    @inbounds return 2*J*state[pos]*( state[i%L+1,j]+state[i,j%L+1]+state[i==1?L:i-1,j]+state[i,j==1?L:j-1] ) + 2*h*state[pos]
end

## Magnetization per spin
function m(state) 
    sum(state)/length(state)
end
    
## Perform one step of the Metropolis algorithm.
## `state` is mutated.
## Arguments:
##  state::Matrix{Int8} Current configuration 
##  beta::Float64       Inverse temperature
##  h::Float64          External field
function metropolis_step!(state::Matrix{Int8},beta,h)
    i = rand(1:length(state))
    dh = dH(state,i,1.,h)
    if dh <= 0 || rand()<exp(-beta*dh)
        state[i] *= -1
    end
    return 0
end

function sweep!(state,n,beta,h)
    for _ in 1:n
        metropolis_step!(state,beta,h)
    end
end
    
## Runs the Metropolis algorithm for a given set of parameters.
## Samples in defined intervals along the Markov-Chain.
## An initial thermal sweep to go to equilibrium may be specified.
## Call like 
##  run_metropolis(50,0.,0.;Tmax=25*10^3*50^2,poll_interval=10*50^2,sweep=10^3*50^2)
##
## Arguments
##  L::Int             Linear system size
##  beta::Float64      Inverse temperature
##  h::Float64         External field
##  Tmax::Int          Number of steps
##  sweep::Int         Length of the initial sweep
##  poll_interval::Int Record the observables every `poll_interval` steps.
##  repeat::Int Restart the chain
    
function run_metropolis(L::Int, beta,h;Tmax::Int=1,sweep::Int=0,poll_interval::Int=1)
    ## Initialise a random state
    function init(L,beta,h)
        state = randomConfiguration(L)
        # Initial sweep to get into the steady state
        sweep!(state,sweep,beta,h)
        return state
    end
    state = init(L,beta,h)
    return _run_metropolis!(state,beta,h,Tmax=Tmax,poll_interval=poll_interval)
end

    
function _run_metropolis!(state::Matrix{Int8},beta,h;Tmax::Int=1,poll_interval::Int=1)
    
    ## Define a matrix in which to record the observables.
    ## One row for each observable, e.g E and m.
    ## Preallocating the matrix gives much better performance than
    ## constructing it on the fly.
    observables = Matrix{Float64}(div(Tmax,poll_interval),2)
    k = 1 #counts the number of samples

    # pick the middle lattice site for the spin correlation
        
    t=0 #number of simulation steps
    r=0 #repeat counter
    @inbounds begin
        while(t<Tmax)
            ## Do the defined no. of steps before
            ## taking a measurement
            sweep!(state,poll_interval,beta,h)
            ## Record observables
            observables[k,1] = H(state,beta,h)
#             observables[k,3] = H(state,beta,h)
            observables[k,2] = m(state)
            ## increment counters
            k+=1
            t+=poll_interval
        end
    end
    
    ## Return statistics about the observables 
    return observables
end
    
end

In [69]:
Tc = 2/log(1+sqrt(2));

In [40]:
obs = DataFrame(Float64,0,5)
names!(obs,[:T,:e,:e2,:m,:m2])

## Temperature range
Ts = linspace(2.,2.6,40)
L = 50

# @everywhere begin state=randomConfiguration($L); sweep!(state,1000*$(L^2),1/$(Ts[1]),0.) end

begin map(pmap(Ts) do T
        @time result = run_metropolis(L, 1/T,0.;Tmax=25*10^3*L^2,sweep=10^3*L^2,poll_interval=10*L^2)
        beta = 1/T
        E = mean(result[:,1])/L^2
        E2 = mean(result[:,1].^2)/L^4
        m = mean(result[:,2])
        m2 = mean(result[:,2].^2)
            
        [T,E,E2,m,m2]'
    end) do row
        push!(obs,row);
end;
obs[:t] = obs[:T]-Tc;
end;

	From worker 4:	  6.424835 seconds (2.81 k allocations: 200.947 KiB)
	From worker 3:	  6.433062 seconds (2.81 k allocations: 200.947 KiB)
	From worker 2:	  6.466876 seconds (2.81 k allocations: 200.947 KiB)
	From worker 3:	  6.373283 seconds (30 allocations: 42.469 KiB)
	From worker 2:	  6.352522 seconds (30 allocations: 42.469 KiB)
	From worker 4:	  6.430829 seconds (30 allocations: 42.469 KiB)
	From worker 3:	  6.371194 seconds (30 allocations: 42.469 KiB)
	From worker 2:	  6.389419 seconds (30 allocations: 42.469 KiB)
	From worker 4:	  6.364715 seconds (30 allocations: 42.469 KiB)
	From worker 2:	  6.319422 seconds (30 allocations: 42.469 KiB)
	From worker 3:	  6.351772 seconds (30 allocations: 42.469 KiB)
	From worker 4:	  6.426129 seconds (30 allocations: 42.469 KiB)
	From worker 2:	  6.346548 seconds (30 allocations: 42.469 KiB)
	From worker 3:	  6.350256 seconds (30 allocations: 42.469 KiB)
	From worker 4:	  6.355293 seconds (30 allocations: 42.469 KiB)
	From worker 3:	  6.31663

In [41]:
obs[:c] = (obs[:e2].-obs[:e].^2)./obs[:T].^2;
obs[:chi] = (obs[:m2].-obs[:m].^2)./obs[:T];

In [4]:
obs = readtable("metropolis02.tsv");

In [40]:
writetable("metropolis02.tsv", obs)

### Analysis

In [58]:
Tcnum = Tc+0.015

2.284185314213022

In [59]:
obs[:t] = obs[:T]-Tcnum;

In [93]:
obs[60:90,[:T,:t]]

,T,t
1,2.2375838926174496,-0.10860142159557244
2,2.2416107382550337,-0.1045745759579884
3,2.2456375838926173,-0.1005477303204048
4,2.2496644295302013,-0.09652088468282076
5,2.2536912751677853,-0.09249403904523672
6,2.257718120805369,-0.08846719340765312
7,2.261744966442953,-0.08444034777006908
8,2.265771812080537,-0.08041350213248503
9,2.2697986577181206,-0.07638665649490144
10,2.2738255033557047,-0.0723598108573174


In [92]:
scatter(obs[:t], abs.(obs[:c]),lab="")
vline!([0.0],lab="Onsager")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.3 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.00006 
 
 
 0.00009 
 
 
 0.00012 
 
 
 0.00015 
 
 
 0.00018 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Onsager

In [64]:
using Interact

In [90]:
@manipulate for offset in -0.2:0.001:0.2
    Tcnum = Tc+offset
    obs[:t] = obs[:T]-Tc-offset
    obs_above = obs[(obs[:t].>0),:];
    obs_below = obs[(obs[:t].<0),:];
    scatter(obs_below[:t], abs.(obs_below[:c]/2.),lab="")
    scatter!(-obs_below[:t], abs.(obs_below[:c]/2.),lab="mirror")

    scatter!(obs_above[:t], abs.(obs_above[:c]),lab="")

    vline!([0.0],lab="Onsager")
end

Interact.Options{:SelectionSlider,Any}(19: "input-6" = 0.0 Any , "offset", 0.0, "0.0", 201, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("-0.2"=>-0.2,"-0.199"=>-0.199,"-0.198"=>-0.198,"-0.197"=>-0.197,"-0.196"=>-0.196,"-0.195"=>-0.195,"-0.194"=>-0.194,"-0.193"=>-0.193,"-0.192"=>-0.192,"-0.191"=>-0.191…), Dict{Any,Any}(Pair{Any,Any}(-0.031, "-0.031"),Pair{Any,Any}(0.117, "0.117"),Pair{Any,Any}(-0.193, "-0.193"),Pair{Any,Any}(-0.081, "-0.081"),Pair{Any,Any}(0.2, "0.2"),Pair{Any,Any}(-0.181, "-0.181"),Pair{Any,Any}(-0.099, "-0.099"),Pair{Any,Any}(-0.191, "-0.191"),Pair{Any,Any}(0.192, "0.192"),Pair{Any,Any}(-0.186, "-0.186")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.000050 
 
 
 0.000075 
 
 
 0.000100 
 
 
 0.000125 
 
 
 0.000150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mirror 
 
 
 
 Onsager

__Split observables in sets above/below $T_c$ __

In [62]:
obs_above = obs[(obs[:t].>0),:];
obs_below = obs[(obs[:t].<0),:];

LoadError: [91mMethodError: no method matching +(::Symbol, ::Float64)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +([91m::Bool[39m, ::T<:AbstractFloat) where T<:AbstractFloat at bool.jl:96
  +([91m::Float64[39m, ::Float64) at float.jl:375
  ...[39m

__Magnetization__

In [14]:
scatter(log.(abs.(obs_below[:t])),log.(abs.(obs_below[:m])), xlabel="log|T-Tc|", legend=false)
#scatter!(log.(abs.(obs_above[:t])),log.(abs.(obs_above[:m])), xlabel="log|T-Tc|", ylabel="log\\chi",legend=false)

plot!(-7.:-1,t->t*1/8+0.08, lw=3)
#plot!(-5.:-1,t->abs(t)*7/4-11)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -7 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 -2.0 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 log|T-Tc|

__Suscpetibility__

In [15]:
scatter(log.(abs.(obs_below[:t])),log.(abs.(obs_below[:chi])), xlabel="log|T-Tc|", legend=true, marker=:auto)
scatter!(log.(abs.(obs_above[:t])),log.(abs.(obs_above[:chi])), xlabel="log|T-Tc|", ylabel="log\\chi", marker=(2,:auto))

plot!(-5.:-1,t->-t*7/4-7.2, lab="above")
plot!(-5.:-1,t->-t*7/4-11., lab="below")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -7 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 -8 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 log|T-Tc| 
 
 
 log 
 
 
 χ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 above 
 
 
 
 below

__Heat capacity__

In [32]:
scatter(log.(abs.(obs_below[:t])),log.(obs_below[:c]), xlabel="log|T-Tc|", legend=false)
scatter!(log.(abs.(obs_above[:t])),log.(obs_above[:c]), xlabel="log|T-Tc|", ylabel="log C",legend=false)
#
plot!(linspace(-5,-1,50),t->log(-t)-10.)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -7 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 -10.0 
 
 
 -9.5 
 
 
 -9.0 
 
 
 -8.5 
 
 
 log|T-Tc| 
 
 
 log C

In [19]:
scatter(log.(abs.(obs_below[:t])),obs_below[:c], xlabel="log|T-Tc|", legend=false)
scatter!(log.(abs.(obs_above[:t])),obs_above[:c], xlabel="log|T-Tc|", ylabel="C",legend=false)
#
# plot!(-5.:-1,t->-t*1/10)
# plot!(-5.:-1,t->-t*1/10+0.04)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -7 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0.00006 
 
 
 0.00009 
 
 
 0.00012 
 
 
 0.00015 
 
 
 0.00018 
 
 
 log|T-Tc| 
 
 
 C

## Real time visualization

In [7]:
using GLVisualize, GLAbstraction, GeometryTypes

INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.6/GLWindow.ji for module GLWindow.
INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.6/GLVisualize.ji for module GLVisualize.


In [8]:
using IterTools,Colors,Reactive,Interact

In [9]:
function color_gen(v0,basecolor)
    map(v0) do x
        if x==1
            RGBA(0.,0.,0.,1.)
        elseif x==-1.
            RGBA(1.,min(1.0,basecolor),0.,1.)
        end
    end
end

color_gen (generic function with 1 method)

__Temperature & field__

In [208]:
temperature

0.44068679350977147

In [87]:
# critical temperature
push!(temperature,1/Tc)

In [10]:
temperature_slider = slider(0.:0.01:10., label="temperature")
display(temperature_slider)
h_slider = slider(-5.:0.1:+5., label="field")
display(h_slider)
a_slider = slider(0.:0.1:2., label="simulation speed")

Interact.Slider{Float64}(1: "input" = 5.0 Float64 , "temperature", 5.0, 0.0:0.01:10.0, "horizontal", true, ".3f", true)

Interact.Slider{Float64}(3: "input-2" = 0.0 Float64 , "field", 0.0, -5.0:0.1:5.0, "horizontal", true, ".3f", true)

Interact.Slider{Float64}(5: "input-3" = 1.0 Float64 , "simulation speed", 1.0, 0.0:0.1:2.0, "horizontal", true, ".3f", true)

In [11]:
temperature=signal(temperature_slider)

5.0

In [12]:
h=signal(h_slider)

0.0

In [13]:
a=signal(a_slider)

1.0

In [58]:
push!(a,10.)

__Prepare the initial state__

In [27]:
# Reset window
try
    empty!(windows)
    close(color_signal)
    close(state_map)
    close(timesignal)
    close(temperature)
catch UndefVarError
end

In [15]:
L=128
config0 = frustratedConfiguration(L);
#config0 = randomConfiguration(L);
for _ in 1:0*L^2*1000
    metropolis_step!(config0,1/value(temperature),0.)
end

__Initialise the OpenGL window__

In [16]:
windows=glscreen(resolution=(800,800));

__Prepare the signals and primitives__

In [17]:
# Reset window
try
    empty!(windows)
    close(color_signal)
    close(state_map)
    close(timesignal)
    close(temperature)
catch UndefVarError
end

if !isopen(windows)
    windows=glscreen(resolution=(800,800))
end

target_fps = 60
timesignal = fps(target_fps)

state_map = map((_,T,h)->for _ in 1:div(value(a)*L^2,target_fps) metropolis_step!(config0,1/T,h) end, timesignal,temperature,h);
color_signal=map(_->reshape(color_gen(config0,0.3),L^2,1)[:,1], timesignal)

#position = Point2f0[Point2f0(800/L*(xi+1/2),800/L*(yi+1/2)) for (xi,yi) in product(0:L-1,0:L-1)]
position = Point3f0[Point3f0(2*xi/L-1.,2*yi/L-1.,0) for (xi,yi) in product(0:L-1,0:L-1)]

#square = HyperRectangle(Vec2f0(0),Vec2f0(800/L));
square = HyperRectangle(Vec3f0(0.),Vec3f0(2/L,2/L,0));

__Visualize the lattice__

In [18]:
lattice = visualize((square,position),color=color_signal);

In [21]:
_view(lattice,windows,camera=:perspective)

In [26]:
eyepos_vec = Vec3f0(0,0,-1)
lookat_vec = Vec3f0(0,0,0)
up_vec = cross(lookat_vec-eyepos_vec,-Vec3f0(1,0,0))

push!(windows.cameras[:perspective].eyeposition, eyepos_vec)
push!(windows.cameras[:perspective].lookat, lookat_vec)
push!(windows.cameras[:perspective].up, up_vec)
push!(windows.cameras[:perspective].fov, 90)

In [230]:
center!(windows)

__Start the render loop__

In [23]:
@async renderloop(windows)

Task (queued) @0x0000000125686650

__Reset the window and signals__

#### Hexagonal lattice

In [112]:
windows=glscreen(resolution=(800,800));

In [217]:
empty!(windows)

In [360]:
L=64

64

In [68]:
center!(windows)

In [175]:
hex_vertices

6-element Array{GeometryTypes.Point{2,Float32},1}:
 Float32[0.108253, 0.0625]  
 Float32[0.108253, -0.0625] 
 Float32[0.0, -0.125]       
 Float32[-0.108253, -0.0625]
 Float32[-0.108253, 0.0625] 
 Float32[0.0, 0.125]        

In [361]:
hex_vertices = map(x->[Point2f0(cos(pi/6),sin(pi/6)) Point2f0(-sin(pi/6),cos(pi/6))]*x, [Point2f0(1,0),
    Point2f0(cos(pi/3),-sin(pi/3)),
    Point2f0(-cos(pi/3),-sin(pi/3)),
    Point2f0(-1,0),
    Point2f0(-cos(pi/3),sin(pi/3)),
    Point2f0(cos(pi/3),sin(pi/3))]/L/Float32(cos(pi/6)))

hex_mesh=GLNormalMesh(hex_vertices)

off = (sqrt(5)-sqrt(3))/2
hex_positions = Point3f0[Point3f0(2*(xi+ifelse(yi%2==0,0.,1/2))/L-1.,2*(yi+1/2+off/2*ifelse(yi==0,0.,-yi))/L-1.,0) for (xi,yi) in product(0:L-1,0:L-1)]
roj_hex = visualize((hex_mesh,hex_positions),color=Signal([RGBA(0.5,1/2*(-1).^(i)+1/2.,0.,1.) for i=0:L^2-1]))

GLAbstraction.Context{GLAbstraction.DeviceUnit}(GLAbstraction.Composable[RenderObject with ID: 54
], 1729: "map(input-1306, input-1305, map(map(input-1298, map(input-1286, input-1287, input-1288, input-1289, input-1290), map(input-1291, input-1292, input-1293, input-1294, input-1295), map(input-1296, input-1297))), input-1304)" = GeometryTypes.HyperRectangle{3,Float32}(Float32[-1.01563, -1.00242, 0.0], Float32[2.01563, 1.75676, 0.0]) GeometryTypes.HyperRectangle{3,Float32} , 1727: "input-1305" = Float32[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0] StaticArrays.SArray{Tuple{4,4},Float32,2,16} )

In [362]:
_view(roj_hex,windows)